# Packages

In [1]:
import ee
import geemap
from geemap.datasets import DATA, get_metadata
from geemap.legends import builtin_legends

In [2]:
ee.Authenticate()
ee.Initialize()

# Using Earth Engine data

Earth Engine objects are server-side objects rather than client-side objects, which means that they are not stored locally in your computer. Similar to video streaming services (e.g. YouTube, Netflix), we can stream geospatial data from Earth Engine on-the-fly without having to download data.
- **Image**: The fundamental raster data type in Earth Engine
- **ImageCollection** A stack of time-series of images
- **Geometry** the fundamental vector type data in Earth Engine
- **Feature** a Geometry with attributes
- **FeatureCollection** A set of features

## Image

Raster data in Earth Engine are represented as **Image** objects. They are composed of one or more bands, and each band has its own name, data type, scale, mask, and projection. Each image has metadata stored as a set of properties.

In [3]:
# Loading EE images
image = ee.Image('USGS/SRTMGL1_003')
image

Name,Description
elevation,Elevation


In [4]:
# Visualizing EE images
m = geemap.Map(center = [21.79, 70.87], zoom = 5)

vis_params = {
    'min': 0,
    'max': 6000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'], # 'terrain'
}

m.addLayer(image, vis_params, 'SRTM')
m

Map(center=[21.79, 70.87], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

## ImageCollection

An `ImageCollection` is a stack or sequence of images. This can be laoded by passing an EE asset ID into the `ImageCollection` constructor. You can find `ImageCollection` IDs in the [EE Data Catalog](https://developers.google.com/earth-engine/datasets).

In [5]:
# Loading image collections
collection = ee.ImageCollection('COPERNICUS/S2_SR')
collection

To visualize an Earth Engine ImageCollection, we need to convert an ImageCollection to an image by compositing all the images in the collection to a single image representing, for example, the min, max, median, or standard deviation.

Let's create a median value image from a collection with the Sentinel-2 surface reflectance collection:

In [6]:
m = geemap.Map()

image = collection.median()

vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

m.set_center(83.277, 17.7009, 12)
m.addLayer(image, vis, 'Sentinel-2')
m

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [7]:
# Filtering image collections
m = geemap.Map()

collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2024-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)

image = collection.median()

vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

m.set_center(83.277, 17.7009, 12)
m.addLayer(image, vis, 'Sentinel-2')
m

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

## FeatureCollection

A **FeatureCollection** is a collection of Features. A FeatureCollection is analogous to a GeoJSON FeatureCollection object, i.e., a collection of features associated with properties/attributes. Data contained in a shapefile can be represented as a FeatureCollection.

The EEDC hosts a variety of vector datasets (e.g. US Census data, country boundaries) as feature collections. You can find feature collection IDs by searching the data catalog.

In [8]:
# Loading feature collections
m = geemap.Map()
fc = ee.FeatureCollection('TIGER/2016/Roads')

m.set_center(-73.9596, 40.7688, 12)
m.addLayer(fc, {}, 'Census Roads')
m

Map(center=[40.7688, -73.9596], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [9]:
# Filtering feature collections
m = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')
fc = states.filter(ee.Filter.eq('NAME', 'Louisiana'))

m.addLayer(fc, {}, 'Louisiana')
m.center_object(fc, 7)
m

Map(center=[30.902226699175234, -91.7982404778954], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
feat = fc.first()
feat.toDictionary()

In [11]:
m = geemap.Map()
fc = states.filter(ee.Filter.inList('NAME', ['California', 'Oregon', 'Washington']))

m.addLayer(fc, {}, 'West Coast')
m.centerObject(fc, 5)
m

Map(center=[41.34849667120548, -120.04086128111338], controls=(WidgetControl(options=['position', 'transparent…

In [12]:
region = m.user_roi

if region is None:
    region = ee.Geometry.BBox(-88.40, 29.88, -77.90, 35.39)

fc = ee.FeatureCollection('TIGER/2018/States').filterBounds(region)

m.addLayer(fc, {}, 'Southeastern US')
m.centerObject(fc, 6)
m

Map(center=[32.864184680063175, -84.06434366110032], controls=(WidgetControl(options=['position', 'transparent…

In [13]:
# Visualizing feature collections
m = geemap.Map(center = [40, -100], zoom = 4)

states = ee.FeatureCollection('TIGER/2018/States')
m.addLayer(states, {}, 'US States')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [14]:
# Visualizing feature collections -- add style
m = geemap.Map(center = [40, -100], zoom = 4)

states = ee.FeatureCollection('TIGER/2018/States')
style = {
    'color': '0000ffff',
    'width': 2,
    'lineType': 'solid',
    'fillColor': 'FF000088'
}
m.addLayer(states.style(**style), {}, 'US States')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [15]:
# Visualizing feature collections -- adding visualization parameters
m = geemap.Map(center = [40, -100], zoom = 4)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'color': '000000',
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
m.add_styled_vector(
    states, column = "NAME", palette = palette, layer_name = "Styled vector", **vis_params
)
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Earth Engine Data Catalog

The Earth Engine Data Catalog hosts a variety of geospatial datasets. As of October 2023, the catalog contains over 1,000 datasets with a total size of over 80 petabytes. Some notable datasets include: Landsat, Sentinel, MODIS, NAIP, etc. For a complete list of datasets in CSV or JSON formats, see the [Earth Engine Datasets List](https://github.com/opengeos/Earth-Engine-Catalog/blob/master/gee_catalog.tsv).

The Earth Engine Data Catalog is searchable. You can search datasets by name, keyword, or tag. For example, enter "elevation" in the search box will filter the catalog to show only datasets containing "elevation" in their name, description, or tags. 52 datasets are returned for this search query. Scroll down the list to find the [NASA SRTM Digital Elevation 30m dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003#description). On each dataset page, you can find the following information, including Dataset Availability, Dataset Provider, Earth Engine Snippet, Tags, Description, Code Example, and more (see {numref}`ch03_gee_srtm`). One important piece of information is the Image/ImageCollection/FeatureCollection ID of each dataset, which is essential for accessing the dataset through the Earth Engine JavaScript or Python APIs.

In [16]:
m = geemap.Map()

dataset_xyz = ee.Image('USGS/SRTMGL1_003')
m.addLayer(dataset_xyz, {}, 'USGS/SRTMGL1_003')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [17]:
m = geemap.Map()
dem = ee.Image('USGS/SRTMGL1_003')

vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

m.addLayer(dem, vis_params, 'SRTM DEM')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [18]:
# Using the datasets module
m = geemap.Map(center = [40, -100], zoom = 4)
dataset = ee.Image(DATA.USGS_GAP_CONUS_2011)

m.addLayer(dataset, {}, 'GAP CONUS')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [19]:
# Using the datasets module -- get_metadata
get_metadata(DATA.USGS_GAP_CONUS_2011)

HTML(value='\n            <html>\n            <body>\n                <h3>USGS GAP CONUS 2011</h3>\n          …

In [20]:
m = geemap.Map()

# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
vizParams = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

# Center the map and display the image.
m.set_center(-122.1899, 37.5010, 10)
# San Francisco Bay
m.add_layer(image, vizParams, 'False color composite')

m

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Exercise - Create cloud-free imagery

In [21]:
m = geemap.Map()

texas = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('NAME', 'Texas'))

texas_style = {
    'color': '1e90ff',
    'width': 2,
    'lineType': 'dotted',
    'fillColor': 'ff475788',
    'pointSize': 10,
    'pointShape': 'circle'
}

# Geometry bounds
texas_bounds = ee.Geometry.BBox(-106.71, 25.54, -93, 37.21)

# Sentinel-2
copernicus = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2022-01-01', '2023-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
    .filterBounds(texas_bounds)
)

cop_image = copernicus.median()

cop_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

m.addLayer(cop_image, cop_vis, 'Copernicus Sentinel-2')


# Landsat 9
landsat = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterDate('2022-01-01', '2023-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
    .filterBounds(texas_bounds)
)

land_image = landsat.median()

land_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

m.addLayer(land_image, land_vis, 'Landsat 9')

m.addLayer(texas.style(**texas_style), {}, 'Texas')
m.centerObject(texas, 6)

m

Map(center=[31.42680331716995, -99.28672202014272], controls=(WidgetControl(options=['position', 'transparent_…

# Visualizing Earth Engine data

## Using the inspector tool

In [22]:
m = geemap.Map(center = [40, -100], zoom = 4)

dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(
    ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
)
states = ee.FeatureCollection('TIGER/2018/States')

vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}

m.addLayer(dem, vis_params, 'SRTM DEM')
m.addLayer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'],
    'min': 20,
    'max': 200,
    'gamma': 2.0
    },
    'Landsat 7'
)
m.addLayer(states, {}, 'US States')
m.add('inspector')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Using the plotting tool

In [23]:
m = geemap.Map(center = [40, -100], zoom = 4)

landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(
    ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
)

landsat_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1.4
}
m.addLayer(landsat7, landsat_vis, 'Landsat')

hyperion = ee.ImageCollection('EO1/HYPERION').filter(
    ee.Filter.date('2016-01-01', '2018-01-01')
)

hyperion_vis = {
    'min': 1000,
    'max': 14000,
    'gamma': 2.5
}
m.addLayer(hyperion, hyperion_vis, 'Hyperion')

m.add_plot_gui()
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [24]:
# Set plotting options for Landsat (Image)
m.set_plot_options(add_marker_cluster = True, overlay = True)

In [25]:
# Set plotting options for Hyperion (ImageCollection)
m.set_plot_options(add_marker_cluster = True, plot_type = 'bar')

## Legends, color bars, and labels

In [26]:
# Built-in legends
for legend in builtin_legends:
    print(legend)

NLCD
ESA_WorldCover
ESRI_LandCover
ESRI_LandCover_TS
Dynamic_World
NWI
MODIS/051/MCD12Q1
MODIS/006/MCD12Q1
GLOBCOVER
JAXA/PALSAR
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL
ALOS_landforms


In [27]:
# Add NLCD WMS layer and legend to map
m = geemap.Map(center = [40, -100], zoom = 4)
m.add_basemap('Esri.WorldImagery')
m.add_basemap('NLCD 2021 CONUS Land Cover')
m.add_legend(builtin_legend = 'NLCD', max_width = '100px', height = '455px')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [28]:
# Add NLCD EE layer and legend to map
m = geemap.Map(center = [40, -100], zoom = 4)
m.add_basemap('Esri.WorldImagery')

nlcd = ee.Image('USGS/NLCD_RELEASES/2021_REL/NLCD/2021')
landcover = nlcd.select('landcover')

m.addLayer(landcover, {}, 'NLCD Land Cover 2021')
m.add_legend(title = 'NLCD Land Cover Classification', builtin_legend = 'NLCD', height = '455px')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [29]:
# Add a custome legend by specifying the colors and labels
m = geemap.Map(add_google_map = False)

keys = ['One', 'Two', 'Three', 'Four', 'etc.']

# Colors can be defined by either hex code or RGB (0-255, 0-255, 0-255)
colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

m.add_legend(keys = keys, colors = colors, position = 'bottomright')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [30]:
# Add a custom legend by specifying a dictionary of colors and labels
m = geemap.Map(center = [40, -100], zoom = 4)
m.add_basemap('Esri.WorldImagery')

legend_dict = {
    '11 Open Water': '466b9f',
    '12 Perennial Ice/Snow': 'd1def8',
    '21 Developed, Open Space': 'dec5c5',
    '22 Developed, Low Intensity': 'd99282',
    '23 Developed, Medium Intensity': 'eb0000',
    '24 Developed High Intensity': 'ab0000',
    '31 Barren Land (Rock/Sand/Clay)': 'b3ac9f',
    '41 Deciduous Forest': '68ab5f',
    '42 Evergreen Forest': '1c5f2c',
    '43 Mixed Forest': 'b5c58f',
    '51 Dwarf Scrub': 'af963c',
    '52 Shrub/Scrub': 'ccb879',
    '71 Grassland/Herbaceous': 'dfdfc2',
    '72 Sedge/Herbaceous': 'd1d182',
    '73 Lichens': 'a3cc51',
    '74 Moss': '82ba9e',
    '81 Pasture/Hay': 'dcd939',
    '82 Cultivated Crops': 'ab6c28',
    '90 Woody Wetlands': 'b8d9eb',
    '95 Emergent Herbaceous Wetlands': '6c9fb8',
}

nlcd = ee.Image('USGS/NLCD_RELEASES/2021_REL/NLCD/2021')
landcover = nlcd.select('landcover')

m.addLayer(landcover, {}, 'NLCD Land Cover 2021')
m.add_legend(title = 'NLCD Land Cover Classification', legend_dict = legend_dict)
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [31]:
# Add a horizontal color bar
m = geemap.Map()

dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}

m.addLayer(dem, vis_params, 'SRTM DEM')
m.add_colorbar(vis_params, label = 'Elevation (m)', layer_name = 'SRTM DEM')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [32]:
# Add a vertical colorbar
m.add_colorbar(
    vis_params,
    label="Elevation (m)",
    layer_name="SRTM DEM",
    orientation="vertical",
    max_width="100px"
)

# Make the color bar background transparent
m.add_colorbar(
    vis_params,
    label="Elevation (m)",
    layer_name="SRTM DEM",
    orientation="vertical",
    max_width="100px",
    transparent_bg=True
)

## Split-panel maps

In [33]:
# Create a split map with basemaps; you can't pan the map currently though
m = geemap.Map()
m.split_map(left_layer = 'Esri.WorldTopoMap', right_layer = 'OpenTopoMap')
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [34]:
# Create a split map with EE layers
m = geemap.Map(center = [40, -100], zoom = 4)

nlcd_2001 = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2001').select('landcover')
nlcd_2021 = ee.Image('USGS/NLCD_RELEASES/2021_REL/NLCD/2021').select('landcover')

left_layer = geemap.ee_tile_layer(nlcd_2001, {}, 'NLCD 2001')
right_layer = geemap.ee_tile_layer(nlcd_2021, {}, 'NLCD 2021')

m.split_map(left_layer, right_layer)
m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Linked Maps

We can create a 2x2 linked map for visualizing Sentinel-2 imagery with different band combinations.

In [35]:
image = (
    ee.ImageCollection('COPERNICUS/S2')
    .filterDate('2018-01-01', '2018-09-30')
    .map(lambda img: img.divide(10000))
    .median()
)

vis_params = [
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3, 'gamma': 1.3},
    {'bands': ['B8', 'B11', 'B4'], 'min': 0, 'max': 0.3, 'gamma': 1.3},
    {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 0.3, 'gamma': 1.3},
    {'bands': ['B12', 'B12', 'B4'], 'min': 0, 'max': 0.3, 'gamma': 1.3},
]

labels = [
    'Natural Color (B4/B3/B2)',
    'Land/Water (B8/B11/B4)',
    'Color Infrared (B8/B4/B3)',
    'Vegetation (B12/B11/B4)'
]

geemap.linked_maps(
    rows = 2,
    cols = 2,
    height = '300px',
    center = [38.4151, 21.2712],
    zoom = 12,
    ee_objects = [image],
    vis_params = vis_params,
    labels = labels,
    label_position = 'topright'
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

## Timeseries inspector

In [36]:
# Check the available years of NLCD
m = geemap.Map(center = [40, -100], zoom = 4)
collection = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD').select('landcover')
vis_params = {'bands': ['landcover']}
years = collection.aggregate_array('system:index').getInfo()
years

['2001', '2004', '2006', '2008', '2011', '2013', '2016', '2019']

In [37]:
# Create a timeseries inspector for NLCD
m.ts_inspector(
    left_ts = collection,
    right_ts = collection,
    left_names = years,
    right_names = years,
    left_vis = vis_params,
    right_vis = vis_params,
    width = '80px'
)
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=…

## Time slider

In [38]:
# Create a map for visualizing MODIS vegetation data
m = geemap.Map()

collection = (
    ee.ImageCollection('MODIS/MCD43A4_006_NDVI')
    .filter(ee.Filter.date('2018-06-01', '2018-07-01'))
    .select('NDVI')
)

vis_params = {
    'min': 0,
    'max': 1,
    'palette': 'ndvi'
}

m.add_time_slider(collection, vis_params, time_interval = 2)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [39]:
# Create a map for visualizing weather data
m = geemap.Map()

collection = (
    ee.ImageCollection('NOAA/GFS0P25')
    .filterDate('2018-12-22', '2018-12-23')
    .limit(24)
    .select('temperature_2m_above_ground')
)

vis_params = {
    'min': -40.0,
    'max': 40.0,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}

labels = [str(n).zfill(2) + ':00' for n in range(24)]
m.add_time_slider(collection, vis_params, labels = labels, time_interval = 1, opacity = 0.8)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [40]:
# Visualizing Sentinel-2 imagery
m = geemap.Map(center = [37.75, -122.45], zoom = 12)

collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(ee.Geometry.Point([-122.45, 37.75]))
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)
)

vis_params = {
    'min': 0,
    'max': 4000,
    'bands': ['B8', 'B4', 'B3']
}

m.add_time_slider(collection, vis_params)
m

Map(center=[37.75, -122.45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Exercise - Create land cover maps with legend

In [41]:
# NLCD Land Cover change in Texas between 2001 and 2019, with legend
m = geemap.Map()
m.add_basemap('Esri.WorldImagery')

texas = (
    ee.FeatureCollection('TIGER/2018/States')
    .filter(ee.Filter.eq('NAME', 'Texas'))
)

nlcd_2001 = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2001').select('landcover').clip(texas)
nlcd_2019 = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019').select('landcover').clip(texas)

left_layer = geemap.ee_tile_layer(nlcd_2001, {}, 'NLCD 2001')
right_layer = geemap.ee_tile_layer(nlcd_2019, {}, 'NLCD 2019')

m.split_map(left_layer, right_layer)
m.add_legend(builtin_legend = 'NLCD')
m.centerObject(texas, 6)
m

Map(center=[31.42680331716995, -99.28672202014272], controls=(ZoomControl(options=['position', 'zoom_in_text',…

# Analyzing Earth Engine Data

## Image descriptive statistics

In [42]:
# Sample Landsat image
m = geemap.Map()

centroid = ee.Geometry.Point([-122.4439, 37.7538])
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(centroid).first()
vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

m.centerObject(centroid, zoom = 8)
m.addLayer(image, vis, 'Landsat-8')
m.addLayer(centroid, {}, 'Centroid')
m

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position', 'transparent…

In [43]:
# Check image properties
image.propertyNames()

In [44]:
# Check image property values
image.toDictionary()

In [45]:
# Get specific image properties
image.get('CLOUD_COVER')

In [46]:
# Get image properties with easy-to-read time format
props = geemap.image_props(image)
props

In [47]:
# Compute image descriptive statistics
stats = geemap.image_stats(image, scale = 20)
stats

## Zonal statistics

### Single image

In [48]:
# Add EE data to map
m = geemap.Map(center = [40, -100], zoom = 4)

# Add NASA SRTM
dem = ee.Image('USGS/SRTMGL1_003')
dem_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
m.addLayer(dem, dem_vis, 'SRTM DEM')

# Add 5-year LANDSAT TOA composite
landsat = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1.4
}
m.addLayer(landsat, landsat_vis, 'Landsat', False)

# Add US Census States
states = ee.FeatureCollection('TIGER/2018/States')
style = {'fillColor': '00000000'}
m.addLayer(states.style(**style), {}, 'US States')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [49]:
# Compute zonal statistics e.g. mean elevation of each state
out_dem_stats = 'data/dem_stats.csv'
geemap.zonal_stats(
    dem, states, out_dem_stats, stat_type = 'MEAN', scale = 1000, return_fc = False
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\dem_stats.csv


In [50]:
geemap.csv_to_df(out_dem_stats).sort_values(by=['mean'], ascending=True)

,mean,STATENS,GEOID,AWATER,LSAD,STUSPS,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
30,13.904218,1779781,10,1399985648,0,DE,10,A,38.998566,5,3,Delaware,-75.441644,G4000,5045925646,0000000000000000000b
1,14.699425,1779809,69,4644252461,0,MP,69,A,14.936784,0,9,Commonwealth of the Northern Mariana Islands,145.601021,G4000,472292529,00000000000000000023
0,15.149698,1802710,78,1550236201,0,VI,78,A,18.326748,0,9,United States Virgin Islands,-64.971251,G4000,348021896,00000000000000000022
27,21.218275,294478,12,31361101223,0,FL,12,A,28.457430,5,3,Florida,-82.409148,G4000,138949136250,00000000000000000001
3,26.543959,1802701,60,1307243754,0,AS,60,A,-14.267159,0,9,American Samoa,-170.668267,G4000,197759063,00000000000000000029
39,30.862419,1629543,22,23753621895,0,LA,22,A,30.863437,7,3,Louisiana,-91.798717,G4000,111897594374,00000000000000000014
2,36.152070,1802705,66,934337453,0,GU,66,A,13.438289,0,9,Guam,144.772949,G4000,543555840,00000000000000000024
34,43.438403,1702382,11,18687198,0,DC,11,A,38.904103,5,3,District of Columbia,-77.017229,G4000,158340391,00000000000000000035
5,57.769871,1219835,44,1323670487,0,RI,44,A,41.597419,1,1,Rhode Island,-71.527272,G4000,2677779902,00000000000000000005
11,65.266327,1779795,34,3544860246,0,NJ,34,A,40.107274,2,1,New Jersey,-74.665201,G4000,19047825980,0000000000000000000e


In [51]:
# Compute the zonal statistics of mean spectral values of each states
out_landsat_stats = 'data/landsat_stats.csv'

geemap.zonal_stats(
    landsat, states, out_landsat_stats, stat_type = 'MEAN', scale = 1000, return_fc = False
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\landsat_stats.csv


In [52]:
geemap.csv_to_df(out_landsat_stats)

,B1,B2,B3,B4,B5,B6_VCID_2,B7,STATENS,GEOID,AWATER,...,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
0,31.388169,21.705126,15.932166,21.327083,14.938834,195.713347,9.025148,1802710,78,1550236201,...,78,A,18.326748,0,9,United States Virgin Islands,-64.971251,G4000,348021896,00000000000000000022
1,28.235328,17.018901,11.455564,15.965257,8.924914,195.315727,5.052182,1779809,69,4644252461,...,69,A,14.936784,0,9,Commonwealth of the Northern Mariana Islands,145.601021,G4000,472292529,00000000000000000023
2,28.668204,20.113093,14.315282,33.956516,19.119983,195.479526,9.265647,1802705,66,934337453,...,66,A,13.438289,0,9,Guam,144.772949,G4000,543555840,00000000000000000024
3,28.647075,18.090196,12.326404,20.628377,9.692754,195.222892,5.489101,1802701,60,1307243754,...,60,A,-14.267159,0,9,American Samoa,-170.668267,G4000,197759063,00000000000000000029
4,28.336682,22.331603,16.056443,50.435245,28.273058,196.526227,12.979382,1779808,72,4922382562,...,72,A,18.217648,0,9,Puerto Rico,-66.410799,G4000,8868896030,00000000000000000031
5,26.693995,20.365608,14.392753,48.919237,26.154328,190.419794,11.823346,1219835,44,1323670487,...,44,A,41.597419,1,1,Rhode Island,-71.527272,G4000,2677779902,00000000000000000005
6,24.182084,19.897618,14.210275,66.181356,32.332016,188.583438,13.340159,1779794,33,1026675248,...,33,A,43.672691,1,1,New Hampshire,-71.584315,G4000,23189413166,00000000000000000007
7,24.753833,20.574505,14.488847,71.927340,34.637324,188.670951,14.045808,1779802,50,1030416650,...,50,A,44.068577,1,1,Vermont,-72.669184,G4000,23874175944,00000000000000000009
8,26.434410,21.109626,16.415626,56.396922,32.885391,189.828376,14.392366,1779780,9,1815617571,...,9,A,41.579864,1,1,Connecticut,-72.746657,G4000,12542497068,0000000000000000000a
9,24.399284,19.500602,13.340003,61.643714,28.630826,188.700606,11.798984,1779787,23,11746549764,...,23,A,45.409284,1,1,Maine,-68.666616,G4000,79887426037,00000000000000000025


### By group

In [53]:
# Area of each land cover type in each state
m = geemap.Map(center = [40, -100], zoom = 4)

# Add NLCD data
dataset = ee.Image('USGS/NLCD_RELEASES/2021_REL/NLCD/2021')
landcover = dataset.select('landcover')
m.addLayer(landcover, {}, 'NLCD 2021')

# Add US Census states
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
m.addLayer(states.style(**style), {}, 'US States')

# Add NLCD legend
m.add_legend(title = 'NLCD Land Cover', builtin_legend = 'NLCD')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [54]:
# Compute zonal statistics by group and convert the unit area from sqm to sqkm
nlcd_stats = 'data/nlcd_stats.csv'

geemap.zonal_stats_by_group(
    landcover, states, nlcd_stats, stat_type = 'SUM', denominator = 1e6, decimal_places = 4
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\nlcd_stats.csv


In [55]:
geemap.csv_to_df(nlcd_stats)

,Class_81,Class_71,Class_82,Class_95,Class_41,Class_52,Class_31,Class_42,Class_21,Class_43,...,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,78,A,18.326748,0,9,United States Virgin Islands,-64.971251,G4000,348021896,00000000000000000022
1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,69,A,14.936784,0,9,Commonwealth of the Northern Mariana Islands,145.601021,G4000,472292529,00000000000000000023
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,66,A,13.438289,0,9,Guam,144.772949,G4000,543555840,00000000000000000024
3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,60,A,-14.267159,0,9,American Samoa,-170.668267,G4000,197759063,00000000000000000029
4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,72,A,18.217648,0,9,Puerto Rico,-66.410799,G4000,8868896030,00000000000000000031
5,81.0853,50.8536,17.4843,31.6379,858.7944,29.3414,27.9410,101.9544,104.9196,401.8067,...,44,A,41.597419,1,1,Rhode Island,-71.527272,G4000,2677779902,00000000000000000005
6,719.4609,121.5805,70.5616,103.7194,5293.1332,177.9115,75.1567,4370.3054,524.1626,9100.6537,...,33,A,43.672691,1,1,New Hampshire,-71.584315,G4000,23189413166,00000000000000000007
7,3141.2259,45.8622,434.7394,114.3151,9588.6768,74.2761,33.1171,3129.4369,387.4514,5400.5034,...,50,A,44.068577,1,1,Vermont,-72.669184,G4000,23874175944,00000000000000000009
8,462.9919,50.0740,214.6323,87.3920,6389.1296,26.3440,32.8023,140.2907,941.2159,1329.8514,...,9,A,41.579864,1,1,Connecticut,-72.746657,G4000,12542497068,0000000000000000000a
9,2241.0497,1539.7298,1062.5933,783.2863,12944.4479,887.9636,116.3240,17930.2106,869.6415,29878.2847,...,23,A,45.409284,1,1,Maine,-68.666616,G4000,79887426037,00000000000000000025


In [56]:
# Calculate the percentage of each land cover type in each state
nlcd_stats2 = 'data/nlcd_stats_pct.csv'

geemap.zonal_stats_by_group(
    landcover, states, nlcd_stats2, stat_type = 'PERCENTAGE', denominator = 1e6, decimal_places = 4
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\nlcd_stats_pct.csv


In [57]:
geemap.csv_to_df(nlcd_stats2)

,Class_81,Class_71,Class_82,Class_95,Class_41,Class_52,Class_31,Class_42,Class_21,Class_43,...,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,78,A,18.326748,0,9,United States Virgin Islands,-64.971251,G4000,348021896,00000000000000000022
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,69,A,14.936784,0,9,Commonwealth of the Northern Mariana Islands,145.601021,G4000,472292529,00000000000000000023
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,66,A,13.438289,0,9,Guam,144.772949,G4000,543555840,00000000000000000024
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,60,A,-14.267159,0,9,American Samoa,-170.668267,G4000,197759063,00000000000000000029
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,72,A,18.217648,0,9,Puerto Rico,-66.410799,G4000,8868896030,00000000000000000031
5,0.020267,0.012711,0.004370,0.007908,0.214658,0.007334,0.006984,0.025484,0.026225,0.100433,...,44,A,41.597419,1,1,Rhode Island,-71.527272,G4000,2677779902,00000000000000000005
6,0.029711,0.005021,0.002914,0.004283,0.218584,0.007347,0.003104,0.180475,0.021646,0.375819,...,33,A,43.672691,1,1,New Hampshire,-71.584315,G4000,23189413166,00000000000000000007
7,0.126136,0.001842,0.017457,0.004590,0.385034,0.002983,0.001330,0.125663,0.015558,0.216858,...,50,A,44.068577,1,1,Vermont,-72.669184,G4000,23874175944,00000000000000000009
8,0.032246,0.003488,0.014949,0.006087,0.444987,0.001835,0.002285,0.009771,0.065553,0.092621,...,9,A,41.579864,1,1,Connecticut,-72.746657,G4000,12542497068,0000000000000000000a
9,0.024458,0.016804,0.011597,0.008549,0.141272,0.009691,0.001270,0.195685,0.009491,0.326083,...,23,A,45.409284,1,1,Maine,-68.666616,G4000,79887426037,00000000000000000025


### Two images

In [58]:
# Similar to zonal statistics by group, but takes an image as zone input instead of a feature collection
m = geemap.Map(center = [40, -100], zoom = 4)
dem = ee.Image('USGS/3DEP/10m')
vis = {
    'min': 0,
    'max': 4000,
    'palette': 'terrain'
}
m.addLayer(dem, vis, 'DEM')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [59]:
landcover = ee.Image("USGS/NLCD_RELEASES/2021_REL/NLCD/2021").select('landcover')
m.addLayer(landcover, {}, 'NLCD 2021')
m.add_legend(title='NLCD Land Cover Classification', builtin_legend='NLCD')

In [60]:
# Compute the mean elevation of each land cover type
stats = geemap.image_stats_by_zone(dem, landcover, reducer = 'MEAN')
stats

,zone,stat
0,11,157.088502
1,12,2087.599387
2,21,199.447195
3,22,235.956287
4,23,245.613319
5,24,179.232739
6,31,1136.430165
7,41,408.416279
8,42,1139.498470
9,43,296.424302


In [61]:
stats.to_csv('data/nlcd_mean.csv', index = False)

In [62]:
# Compute the standard deviation of each land cover type
geemap.image_stats_by_zone(
    dem, landcover, out_csv = 'data/nlcd_std.csv', reducer = 'STD'
)

'data/nlcd_std.csv'

In [63]:
geemap.csv_to_df('data/nlcd_std.csv')

,zone,stat
0,11,254.810430
1,12,469.130053
2,21,315.044529
3,22,305.072925
4,23,358.639694
5,24,263.305329
6,31,821.489416
7,41,427.026233
8,42,969.532187
9,43,246.360083


## Exercise - Zonal Statistics

In [64]:
# Find out which state has highest mean temperature on US on June 28, 2023
m = geemap.Map(center = [40, -100], zoom = 4)

# Temperature
collection = (
    ee.ImageCollection('NOAA/GFS0P25')
    .filterDate('2018-06-23', '2018-06-24')
    .select('temperature_2m_above_ground')
    .mean()
)

vis_params = {
    'min': -40.0,
    'max': 35.0,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
m.addLayer(collection, vis_params, 'Mean Temperature')

# States
states = ee.FeatureCollection('TIGER/2018/States')
style = {'fillColor': '00000000'}
m.addLayer(states.style(**style), {}, 'US States')
m.add_colorbar(vis_params, label = 'Mean Temperature', layer_name = 'Mean Temperature')

m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [65]:
collection

In [66]:
temp_stats = 'data/temp_stats.csv'

geemap.zonal_stats(
    collection, states, temp_stats, stat_type = 'MEAN', scale = 1000, return_fc = False
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\temp_stats.csv


In [67]:
geemap.csv_to_df(temp_stats).sort_values(by = ['mean'], ascending = False)

,mean,STATENS,GEOID,AWATER,LSAD,STUSPS,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
40,29.959788,1779801,48,19006305260,0,TX,48,A,31.434703,7,3,Texas,-99.281824,G4000,676653171537,00000000000000000019
41,29.690795,1102857,40,3374587997,0,OK,40,A,35.590051,7,3,Oklahoma,-97.486815,G4000,177662925723,0000000000000000001c
1,29.019253,1779809,69,4644252461,0,MP,69,A,14.936784,0,9,Commonwealth of the Northern Mariana Islands,145.601021,G4000,472292529,00000000000000000023
27,28.929371,294478,12,31361101223,0,FL,12,A,28.457430,5,3,Florida,-82.409148,G4000,138949136250,00000000000000000001
39,28.785727,1629543,22,23753621895,0,LA,22,A,30.863437,7,3,Louisiana,-91.798717,G4000,111897594374,00000000000000000014
23,28.597778,481813,20,1344141205,0,KS,20,A,38.498546,4,2,Kansas,-98.383430,G4000,211755344060,0000000000000000002f
50,28.551270,1779777,4,1027337603,0,AZ,4,A,34.203936,8,4,Arizona,-111.606357,G4000,294198551143,00000000000000000037
38,28.501646,1779790,28,3926919758,0,MS,28,A,32.686471,6,3,Mississippi,-89.656138,G4000,121533519481,0000000000000000002c
2,28.419017,1802705,66,934337453,0,GU,66,A,13.438289,0,9,Guam,144.772949,G4000,543555840,00000000000000000024
42,28.353616,68085,5,2962859592,0,AR,5,A,34.895526,7,3,Arkansas,-92.444626,G4000,134768872727,0000000000000000002b


## Coordinate grids and fishnets

### Creating coordinate grids

In [68]:
# Create a latitudinal grid with a 5-degree interval
lat_grid = geemap.latitude_grid(step = 5.0, west = -180, east = 180, south = -85, north = 85)

m = geemap.Map()
style = {'fillColor': '00000000'}
m.addLayer(lat_grid.style(**style), {}, 'Latitude Grid')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [69]:
coord_grid = geemap.ee_to_df(lat_grid)
coord_grid.head()

,east,north,south,west
0,180,-80,-85,-180
1,180,-75,-80,-180
2,180,-70,-75,-180
3,180,-65,-70,-180
4,180,-60,-65,-180


In [70]:
# Create a longitudinal grid with a 5-degree interval
lon_grid = geemap.longitude_grid(step=5.0, west=-180, east=180, south=-85, north=85)

m = geemap.Map()
style = {'fillColor': '00000000'}
m.add_layer(lon_grid.style(**style), {}, 'Longitude Grid')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [71]:
# Create a rectangular grid with a 10-degree interval
grid = geemap.latlon_grid(
    lat_step = 10, lon_step = 10,
    west = -180, east = 180, south = -85, north = 85
)

m = geemap.Map()
style = {'fillColor': '00000000'}
m.addLayer(grid.style(**style), {}, 'Coordinate Grid')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Creating fishnets

In [72]:
# Use drawing tools to draw a polygon on the map above (or use the default polygon)
m = geemap.Map()

roi = m.user_roi

if roi is None:
    roi = ee.Geometry.BBox(-112.8089, 33.7306, -88.5951, 46.6244)
    m.add_layer(roi, {}, 'ROI')

m.center_object(roi)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [73]:
# Create a fishnet based on a user-drawn polygon with a 2-degree interval
fishnet = geemap.fishnet(roi, h_interval = 2.0, v_interval = 2.0, delta = 1)
style = {'color': 'blue', 'fillColor': '00000000'}
m.addLayer(fishnet.style(**style), {}, 'Fishnet')

In [74]:
# New map with polygon
m = geemap.Map()

roi = m.user_roi

if roi is None:
    roi = ee.Geometry.Polygon(
        [
            [
                [-64.602356, -1.127399],
                [-68.821106, -12.625598],
                [-60.647278, -22.498601],
                [-47.815247, -21.111406],
                [-43.860168, -8.913564],
                [-54.582825, -0.775886],
                [-60.823059, 0.454555],
                [-64.602356, -1.127399],
            ]
        ]
    )
    m.add_layer(roi, {}, 'ROI')

m.center_object(roi)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [75]:
# Create a fishnet based on a user-drawn polygon with specified numbers or rows and columns
fishnet = geemap.fishnet(roi ,row = 6, cols = 8, delta = 1)
style = {
    'color': 'blue',
    'fillColor': '00000000'
}
m.addLayer(fishnet.style(**style), {}, 'Fishnet')

## Land use and land cover change analysis

### Forest cover mapping

We will use the [Hansen Global Forest Change v1.10 (2000-2022) dataset](https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2022_v1_10).

In [76]:
dataset = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')
dataset.bandNames()

In [77]:
# Select the imagery for 2000
m = geemap.Map()
first_bands = ['first_b50', 'first_b40', 'first_b30']
first_image = dataset.select(first_bands)
m.addLayer(first_image, {'bands': first_bands, 'gamma': 1.5}, 'Landsat 2000')

# Select the imagery for 2023
last_bands = ['last_b50', 'last_b40', 'last_b30']
last_image = dataset.select(last_bands)
m.addLayer(last_image, {'bands': last_bands, 'gamma': 1.5}, 'Landsat 2023')

# Select tree cover imagery for 2000
treecover = dataset.select(['treecover2000'])
treeCoverVisParams = {
    'min': 0,
    'max': 100,
    'palette': ['black', 'green'],
}
name = 'Tree cover (%)'
m.addLayer(treecover, treeCoverVisParams, name)
m.add_colorbar(treeCoverVisParams, label = name, layer_name = name)
m.add('layer_manager')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [78]:
# Extract tree cover 2000 by threshold
threshold = 10
treecover_bin = treecover.gte(threshold).selfMask()
treeVisParams = {'palette': ['brown']}
m.addLayer(treecover_bin, treeVisParams, 'Tree cover bin')

### Forest loss and gain mapping

In [79]:
# Visualize forest loss
m = geemap.Map()
m.add_basemap('Esri.WorldImagery')
treeloss_year = dataset.select(['lossyear'])
treeLossVisParams = {
    'min': 0,
    'max': 22,
    'palette': ['yellow', 'red']
}

layer_name = 'Tree loss year'
m.addLayer(treeloss_year, treeLossVisParams, layer_name)
m.add_colorbar(treeLossVisParams, label = layer_name, layer_name = layer_name)
m.add('layer_manager')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [80]:
# Compare forest loss and gain
m = geemap.Map()
m.add_basemap('Esri.WorldImagery')

treeloss = dataset.select(['loss']).selfMask()
treegain = dataset.select(['gain']).selfMask()

m.addLayer(treeloss, {'palette': 'red'}, 'Tree loss')
m.addLayer(treegain, {'palette': 'green'}, 'Tree gain')
m.add('layer_manager')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Zonal statistics by country

Compute zonal statistics to find out which country has the largest forest area in 2000.

In [81]:
# Add a country boundary to the map
m = geemap.Map()
countries = ee.FeatureCollection(geemap.examples.get_ee_path('countries'))
style = {
    'color': '#000000ff',
    'fillColor': ' #00000000'
}

m.addLayer(countries.style(**style), {}, 'Countries')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [82]:
# Compute zonal statistics by country
geemap.zonal_stats(
    treecover_bin, countries, 'data/forest_cover.csv', stat_type = 'SUM', denominator = 1e6, scale = 1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\forest_cover.csv


In [83]:
# Create a pie chart to visualize the forest area by country
geemap.pie_chart(
    'data/forest_cover.csv', names = 'NAME', values = 'sum', max_rows = 20, height = 400
)

In [84]:
# Create a bar chart to visualize the forest area by country
geemap.bar_chart(
    'data/forest_cover.csv', x = 'NAME', y = 'sum', max_rows = 20, x_label = 'Country', y_label = 'Forest area (km2)'
)

In [85]:
# Calculate the forest loss area by country
geemap.zonal_stats(
    treeloss.gt(0), countries, 'data/treeloss.csv', stat_type = 'SUM', denominator = 1e6, scale = 1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\treeloss.csv


In [86]:
# Create a pie chart to visualize the forest loss area by country
geemap.pie_chart(
    'data/treeloss.csv', names = 'NAME', values = 'sum', max_rows = 20, height = 600
)

In [87]:
# Create a bar chart to visualize the forest loss area by country
geemap.bar_chart(
    'data/treeloss.csv', x = 'NAME', y = 'sum', max_rows = 20, x_label = 'Country', y_label = 'Forest loss area (km2)'
)

## Exercise - Analyzing forest cover gain and loss

Find out which US state has the largest forest gain and loss between 2000 and 2022. Create pie charts and bar charts.

In [88]:
m = geemap.Map(center = [40, -100], zoom = 4)
m.add_basemap('Esri.WorldImagery')

states = ee.FeatureCollection('TIGER/2018/States')
dataset = ee.Image('UMD/hansen/global_forest_change_2022_v1_10').clip(states)

# 2000
first_bands = ['first_b50', 'first_b40', 'first_b30']
first_image = dataset.select(first_bands)
m.add_layer(first_image, {'bands': first_bands, 'gamma': 1.5}, 'Landsat 2000')

# 2022
last_bands = ['last_b50', 'last_b40', 'last_b30']
last_image = dataset.select(last_bands)
m.add_layer(last_image, {'bands': last_bands, 'gamma': 1.5}, 'Landsat 2022')

# Tree cover
treecover = dataset.select(['treecover2000'])
treeCoverVisParam = {'min': 0, 'max': 100, 'palette': ['black', 'green']}
name = 'Tree cover (%)'
m.add_layer(treecover, treeCoverVisParam, name)
m.add_colorbar(treeCoverVisParam, label=name, layer_name=name)

# Forest Loss and Gain
treeloss = dataset.select(['loss']).selfMask()
treegain = dataset.select(['gain']).selfMask()
m.addLayer(treeloss, {'palette': 'red'}, 'Tree loss')
m.addLayer(treegain, {'palette': 'yellow'}, 'Tree gain')

m.add('layer_manager')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [89]:
# Forest Loss Area by state
tl = 'data/state_treeloss.csv'

geemap.zonal_stats(
    treeloss.gt(0), states, tl, stat_type = 'SUM', denominator = 1e6, scale = 1000
)
geemap.csv_to_df(tl).sort_values(by = ['sum'], ascending = False)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\state_treeloss.csv


,sum,STATENS,GEOID,AWATER,LSAD,STUSPS,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
55,254828.372549,1785533,2,245481577452,0,AK,2,A,63.347356,9,4,Alaska,-152.839733,G4000,1478839695958,00000000000000000028
31,86655.015686,1705317,13,4422936154,0,GA,13,A,32.629579,5,3,Georgia,-83.423511,G4000,149482048342,00000000000000000015
35,80741.239216,1779775,1,4593327154,0,AL,1,A,32.739632,6,3,Alabama,-86.843459,G4000,131174048583,00000000000000000016
51,72899.411765,1779778,6,20463871877,0,CA,6,A,37.155177,9,4,California,-119.543418,G4000,403503931312,0000000000000000000d
38,62994.662745,1779790,28,3926919758,0,MS,28,A,32.686471,6,3,Mississippi,-89.656138,G4000,121533519481,0000000000000000002c
52,58449.568627,1155107,41,6192386935,0,OR,41,A,43.971713,9,4,Oregon,-120.622958,G4000,248606993270,00000000000000000010
53,51115.407843,1779804,53,12559278850,0,WA,53,A,47.407324,9,4,Washington,-120.575800,G4000,172112588220,00000000000000000013
27,48651.192157,294478,12,31361101223,0,FL,12,A,28.457430,5,3,Florida,-82.409148,G4000,138949136250,00000000000000000001
29,48499.176471,1027616,37,13466071395,0,NC,37,A,35.539710,5,3,North Carolina,-79.130864,G4000,125923656064,00000000000000000008
32,47482.164706,1779799,45,5075218778,0,SC,45,A,33.874178,5,3,South Carolina,-80.854264,G4000,77864918488,0000000000000000001b


In [90]:
# Loss Area - Pie Chart
geemap.pie_chart(
    tl, names = 'NAME', values = 'sum', max_rows = 20, height = 400
)

In [91]:
# Loss Area - Bar Chart
geemap.bar_chart(
    tl, x = 'NAME', y = 'sum', max_rows = 20, x_label = 'State', y_label = 'Forest loss area (km2)'
)

In [92]:
# Forest Area Gain by state
tg = 'data/state_treegain.csv'

geemap.zonal_stats(
    treegain.gt(0), states, tg, stat_type = 'SUM', denominator = 1e6, scale = 1000
)

geemap.csv_to_df(tg).sort_values(by = 'sum', ascending = False)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\state_treegain.csv


,sum,STATENS,GEOID,AWATER,LSAD,STUSPS,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
35,39702.811765,1779775,1,4593327154,0,AL,1,A,32.739632,6,3,Alabama,-86.843459,G4000,131174048583,00000000000000000016
38,32781.047059,1779790,28,3926919758,0,MS,28,A,32.686471,6,3,Mississippi,-89.656138,G4000,121533519481,0000000000000000002c
31,32593.074510,1705317,13,4422936154,0,GA,13,A,32.629579,5,3,Georgia,-83.423511,G4000,149482048342,00000000000000000015
39,25862.168627,1629543,22,23753621895,0,LA,22,A,30.863437,7,3,Louisiana,-91.798717,G4000,111897594374,00000000000000000014
27,22606.380392,294478,12,31361101223,0,FL,12,A,28.457430,5,3,Florida,-82.409148,G4000,138949136250,00000000000000000001
42,19760.509804,68085,5,2962859592,0,AR,5,A,34.895526,7,3,Arkansas,-92.444626,G4000,134768872727,0000000000000000002b
52,19570.984314,1155107,41,6192386935,0,OR,41,A,43.971713,9,4,Oregon,-120.622958,G4000,248606993270,00000000000000000010
40,17705.870588,1779801,48,19006305260,0,TX,48,A,31.434703,7,3,Texas,-99.281824,G4000,676653171537,00000000000000000019
32,17144.023529,1779799,45,5075218778,0,SC,45,A,33.874178,5,3,South Carolina,-80.854264,G4000,77864918488,0000000000000000001b
29,14915.588235,1027616,37,13466071395,0,NC,37,A,35.539710,5,3,North Carolina,-79.130864,G4000,125923656064,00000000000000000008


In [93]:
# Gain Area - Pie Chart
geemap.pie_chart(
    tg, names = 'NAME', values = 'sum', max_rows = 20, height = 500
)

In [94]:
# Gain Area - Bar Chart
geemap.bar_chart(
    tg, x = 'NAME', y = 'sum', max_rows = 20, x_label = 'State', y_label = 'Forest gain area (km2)'
)

# Exporting Earth Engine Data

## Exporting images

In [95]:
# Add a Landsat image
m = geemap.Map()

image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318').select(['B5', 'B4', 'B3'])

vis_params = {
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

m.centerObject(image)
m.addLayer(image, vis_params, 'Landsat')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [96]:
# Add a rectangle
region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252)
fc = ee.FeatureCollection(region)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
m.addLayer(fc.style(**style), {}, 'ROI')

# Export to local drive
geemap.ee_export_image(
    image, filename = 'data/landsat.tif', scale = 30, region = region
)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\landsat.tif


In [97]:
# Check image projection
projection = image.select(0).projection().getInfo()
projection

{'type': 'Projection',
 'crs': 'EPSG:32610',
 'transform': [30, 0, 460785, 0, -30, 4264215]}

In [98]:
crs = projection['crs']
crs_transform = projection['transform']

# Specify region, crs, and crs_transform
geemap.ee_export_image(
    image, filename = 'data/landsat_crs.tif', crs = crs, crs_transform = crs_transform, region = region
)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\landsat_crs.tif


## Exporting image collections

In [99]:
point = ee.Geometry.Point(-99.2222, 48.7816)
collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(point)
    .filterDate('2008-01-01', '2018-01-01')
    .filter(ee.Filter.listContains('system:band_names', 'N'))
)
collection.aggregate_array('system:index')

In [100]:
# To local drive
geemap.ee_export_image_collection(
    collection, out_dir = 'data/naip_ic.tif', scale = 10
)

Total number of images: 7

Exporting 1/7: data/naip_ic.tif\m_4809915_sw_14_1_20090823.tif
Generating URL ...
Please wait ...
An error occurred while downloading.
Expecting value: line 1 column 1 (char 0)


## Exporting feature collections

In [101]:
m = geemap.Map()

fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Germany'))

m.addLayer(fc, {}, 'Germany')
m.centerObject(fc)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [102]:
# Shapefile
geemap.ee_to_shp(fc, filename = 'data/Germany.shp', selectors = None)

# Vector / Shapefile2
geemap.ee_export_vector(fc, filename = 'data/Germany2.shp')

# GeoJSON
geemap.ee_to_geojson(fc, filename = 'data/Germany.geojson')

# csv
geemap.ee_to_csv(fc, filename = 'data/Germany.csv')

Using format GEOPANDAS_GEODATAFRAME requires geopandas.
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\Germany2.shp


In [103]:
# # GeoDataFrame
# gdf = geemap.ee_to_gdf(fc)
# gdf

In [104]:
# DataFrame
df = geemap.ee_to_df(fc)
df

,abbreviati,country_co,country_na,wld_rgn
0,Ger.,GM,Germany,Europe


## Exercise - Exporting Images by a Fishnet

Create a fishnet with a 4-degree interval based on the extent of `[-112.5439, 34.0891, -85.0342, 49.6858]`. Use the fishnet to download the Landsat 7 image tiles by the fishnet using the `geemap.download_ee_image_tiles()` and `geemap.download_ee_image_tiles_parallel()` functions. 

In [105]:
m = geemap.Map(center=[40, -100], zoom = 4)
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(['B4', 'B3', 'B2'])
m.add_layer(image, {'min': 20, 'max': 200, 'gamma': 2.0}, 'Landsat')

region = ee.Geometry.BBox(-112.5439, 34.0891, -85.0342, 49.6858)
# m.addLayer(region, {}, 'ROI')
m.center_object(region)

fishnet = geemap.fishnet(region, h_interval = 4, v_interval = 4, delta = 1)
style = {'color': 'yellow', 'fillColor': '00000000'}
m.addLayer(fishnet.style(**style), {}, 'Fishnet')

m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [106]:
geemap.download_ee_image_tiles(image, fishnet, out_dir = 'data/fishnet_exercise.tif', scale = 1000)

01.tif: |          | 0.00/598k (raw) [  0.0%] in 00:00 (eta:     ?)

02.tif: |          | 0.00/599k (raw) [  0.0%] in 00:00 (eta:     ?)

03.tif: |          | 0.00/598k (raw) [  0.0%] in 00:00 (eta:     ?)

04.tif: |          | 0.00/598k (raw) [  0.0%] in 00:00 (eta:     ?)

05.tif: |          | 0.00/599k (raw) [  0.0%] in 00:00 (eta:     ?)

...

25.tif: |          | 0.00/597k (raw) [  0.0%] in 00:00 (eta:     ?)

26.tif: |          | 0.00/598k (raw) [  0.0%] in 00:00 (eta:     ?)

27.tif: |          | 0.00/597k (raw) [  0.0%] in 00:00 (eta:     ?)

28.tif: |          | 0.00/597k (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 28 tiles in 99.32607412338257 seconds.


# Create satellite timelapse animatiosn

## Creating satellite timeseries

In [107]:
# Use the Harmonized Sentinel-2 MSI dataset
collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
collection

In [108]:
# Specfy the location of interest and date range
start_date = '2016-01-01'
end_date = '2023-12-31'
region = ee.Geometry.BBox(-122.5549, 37.6968, -122.3446, 37.8111)
region

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -122.5549,
              37.8111
            ],
            [
              -122.5549,
              37.6968
            ],
            [
              -122.34460000000001,
              37.6968
            ],
            [
              -122.34460000000001,
              37.8111
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

In [109]:
# Create an annual composite
images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency = 'year', reducer = 'median'
)
images

In [110]:
# Display the timeseries
m = geemap.Map()

vis_params = {
    'min': 0,
    'max': 4000,
    'bands': ['B8', 'B4', 'B3']
}

labels = [str(y) for y in range(2016, 2024)]
m.addLayer(images, vis_params, 'Sentinel-2', False)
m.add_time_slider(images, vis_params, time_interval = 2, labels = labels)
m.centerObject(region)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Exercise - Creating timelapse animations

Use the geemap timelapse GUI to create a timelapse animation for any location of your choice.

In [3]:
m = geemap.Map()
m.add_gui('timelapse')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…